In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/My\ Drive/PRML-2020/Data\ Contest

/content/drive/My Drive/PRML-2020/Data Contest


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
from imblearn.over_sampling import SMOTE, SVMSMOTE, BorderlineSMOTE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
data = pd.read_csv('./Required data/train_complete.csv')
data.drop(['Unnamed: 0', 'host_id'], axis=1, inplace=True)
display(data)

,biker_id,tour_id,invited,like,dislike,bornIn,time_zone,tour_latitude,tour_longitude,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,w13,w14,w15,w16,w17,w18,w19,w20,w21,w22,w23,w24,w25,w26,w27,w28,w29,w30,w31,...,w74,w75,w76,w77,w78,w79,w80,w81,w82,w83,w84,w85,w86,w87,w88,w89,w90,w91,w92,w93,w94,w95,w96,w97,w98,w99,w100,w_other,tour_day,tour_month,tour_year,member_since_day,member_since_month,member_since_year,num_friends_going,num_friends_not_going,num_friends_maybe,num_friends_invited,gender_female,gender_male
0,DA44012,QY18771225,0,0,0,1990,480.0,0.000,0.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,10,2012,2,10,2012,0,3,0,3,0,1
1,DA44012,QU02284248,0,0,0,1990,480.0,-7.767,110.363,2,0,0,0,2,3,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,3,10,2012,2,10,2012,0,0,0,1,0,1
2,DA44012,RU29072432,0,1,0,1990,480.0,3.567,98.650,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,37,26,10,2012,2,10,2012,0,0,0,2,0,1
3,DA44012,SP72478280,0,0,0,1990,480.0,34.017,71.583,1,2,1,1,0,0,2,0,0,1,1,1,2,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3,0,56,6,10,2012,2,10,2012,1,0,0,0,0,1
4,DA44012,QS90707377,0,0,0,1990,480.0,3.607,98.653,2,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,79,6,10,2012,2,10,2012,0,0,0,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13861,EC93103086,RW50873665,0,0,0,1993,420.0,-6.973,107.633,1,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,80,8,12,2012,28,10,2012,0,0,1,0,1,0
13862,EC93103086,TP84655790,0,0,0,1993,420.0,0.000,0.000,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,80,9,12,2012,28,10,2012,1,0,0,1,1,0
13863,EC93103086,UY8708806,0,0,0,1993,420.0,-7.973,112.609,3,2,0,1,1,0,0,0,1,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,60,9,12,2012,28,10,2012,1,1,1,3,1,0
13864,EC93103086,VP4179853,0,0,0,1993,420.0,-8.167,113.700,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,12,12,2012,28,10,2012,0,1,1,0,1,0


# Score and cross-validation function

In [5]:
def MAP_Score(y_ord, y_pred_ord):
    MAP = []
    for y_, y_pred in zip(y_ord, y_pred_ord):
        cor = 0
        score = 0
        for i in range(len(y_pred)):
            if y_pred[i] in y_:
                cor += 1
                score += cor/(i+1)
        MAP.append(score/len(y_))
        
    return np.mean(MAP)

a = [[2,4]]
b = [[2,3,5,4,1]]
MAP_Score(a, b)

0.75

In [6]:
def cross_validation(model, data, cls_wgt):
    all_ids = shuffle(data['biker_id'].unique())
    num_ids = len(all_ids)
    MAP_score = []
    
    kfold = KFold(n_splits=5)
    for t_indx, v_indx in kfold.split(all_ids):
        train_ids = all_ids[t_indx]
        val_ids = all_ids[v_indx]
        
        train_indx, val_indx = [], []
        for id in all_ids:
            if id in train_ids:
                train_indx.extend(np.where(data['biker_id'] == id)[0])
            else:
                val_indx.extend(np.where(data['biker_id'] == id)[0])
            
        train_data = data.iloc[train_indx]
        train_bikers = train_data['biker_id']
        train_tour = train_data['tour_id']
        train_data.drop(['biker_id', 'tour_id'], axis=1, inplace=True)
        
        val_data = data.iloc[val_indx]
        val_bikers = val_data['biker_id']
        val_tour = val_data['tour_id']
        val_data.drop(['biker_id', 'tour_id'], axis=1, inplace=True)
        
        X_train = train_data.drop(['like', 'dislike'], axis=1).to_numpy()
        X_val = val_data.drop(['like', 'dislike'], axis=1).to_numpy()
        y_train = train_data['like']
        y_val = val_data['like']
        
        oversample = BorderlineSMOTE(sampling_strategy={0:9000, 1:int(9000*cls_wgt)})
        X_train, y_train = oversample.fit_resample(X_train, y_train)
        
        y_val_ord = []
        y_val_prob = []
        for id in val_ids:
            like = []
            dt = val_data[data['biker_id'] == id]
            for idx, lk in enumerate(dt['like']):
                if lk:
                    like.append(idx)
            y_val_prob.append(list(dt['like']))
            y_val_ord.append(like)
        
        sclr = StandardScaler()
        X_train_scld = sclr.fit_transform(X_train)
        X_val_scld = sclr.transform(X_val)
        
        model.fit(X_train_scld, y_train)
        y_train_pred = model.predict(X_train_scld)
        y_val_pred = model.predict(X_val_scld)

        y_val_pred = model.predict(X_val_scld)
        y_val_pred_prob = model.predict_proba(X_val_scld)
        
        y_val_pred_ord = []
        y_val_prd_prob = []
        
        for id in val_ids:
            indx = np.where(val_bikers==id)[0]
            pred = np.flip(np.argsort(y_val_pred_prob[indx, 1]))
            prob = y_val_pred_prob[indx, 1]
            
            y_val_pred_ord.append(pred)
            y_val_prd_prob.append(prob)
            
        val_map = MAP_Score(y_val_ord, y_val_pred_ord)
        MAP_score.append(val_map)
        
    return np.mean(MAP_score)

# RandomForestClassifier with 200 trees

In [ ]:
MAP_score = []
features = {}

n_est = 200
for msl in tqdm(np.linspace(1, 15, 8, dtype=np.int32)):
    for max_features in np.linspace(0.5, 0.9, 5):
        for max_samples in np.linspace(0.5, 0.9, 5):
            for cls_wgt in np.linspace(0.5, 1.5, 5):
                print('===========================================================================')
                clf = RandomForestClassifier(n_estimators=n_est, max_features=max_features,
                                             max_samples=max_samples,  min_samples_leaf=msl)
                score = cross_validation(clf, data, cls_wgt)
                MAP_score.append(score)
                features[str(score)] = [cls_wgt, n_est, msl, max_features, max_samples]
                print('Class weight : {0:.2f}, Num estimators : {1}, Min samples split : {2}, Max features : {3}, Max samples : {4}, MAP score : {5:.3f}'
                      .format(cls_wgt, n_est, msl, max_features, max_samples, score))

Class weight : 0.50, Num estimators : 200, Min samples split : 1, Max features : 0.5, Max samples : 0.5, MAP score : 0.615
Class weight : 0.75, Num estimators : 200, Min samples split : 1, Max features : 0.5, Max samples : 0.5, MAP score : 0.605
Class weight : 1.00, Num estimators : 200, Min samples split : 1, Max features : 0.5, Max samples : 0.5, MAP score : 0.603
Class weight : 1.25, Num estimators : 200, Min samples split : 1, Max features : 0.5, Max samples : 0.5, MAP score : 0.603
Class weight : 1.50, Num estimators : 200, Min samples split : 1, Max features : 0.5, Max samples : 0.5, MAP score : 0.606
Class weight : 0.50, Num estimators : 200, Min samples split : 1, Max features : 0.5, Max samples : 0.6, MAP score : 0.614
Class weight : 0.75, Num estimators : 200, Min samples split : 1, Max features : 0.5, Max samples : 0.6, MAP score : 0.608
Class weight : 1.00, Num estimators : 200, Min samples split : 1, Max features : 0.5, Max samples : 0.6, MAP score : 0.610
Class weight : 1

In [ ]:
max_map = max(MAP_score)
feat = features[str(max_map)]
max_map, feat

# GradientBoostingClassifier with 200 trees

In [ ]:
MAP_score = []
features = {}

n_est = 200
for cls_wgt in tqdm(np.linspace(0.5, 1.5, 5)):
    for lr in np.linspace(0.1, 0.5, 3):
        for max_features in np.linspace(0.5, 0.9, 5):
            for max_samples in np.linspace(0.5, 0.9, 5):
                for max_depth in np.linspace(10, 50, 5, dtype=np.int32):

                    print('===========================================================================')
                    clf = XGBClassifier(n_estimators=n_est, max_features=max_features,
                                        subsample=max_samples, max_depth=max_depth, 
                                        learning_rate=lr)
                    score = cross_validation(clf, data, cls_wgt)
                    MAP_score.append(score)
                    features[str(score)] = [cls_wgt, n_est, max_depth, max_features, max_samples]
                    print('Class weight : {0:.1f}, Num estimators : {1}, Max depth : {2}, Max features : {3:.2f}, Max samples : {4:.2f}, Learning rate : {5:.1f}, MAP score : {6:.3f}'
                        .format(cls_wgt, n_est, max_depth, max_features, max_samples, lr, score))

In [ ]:
max_map = max(MAP_score)
feat = features[str(max_map)]
max_map, feat